In [1]:
"""Creating tools, using the command shell."""

'Creating tools, using the command shell.'

# Создание инструментов с помощью командной оболочки

<a href="https://dfedorov.spb.ru/pandas/#14">К оглавлению курса</a> </p> <br>

Сильная сторона [UNIX-оболочки](https://habr.com/ru/company/ruvds/blog/325522/) заключается в том, что она позволяет комбинировать программы для создания [конвейеров](https://ru.wikipedia.org/wiki/%D0%9A%D0%BE%D0%BD%D0%B2%D0%B5%D0%B9%D0%B5%D1%80_(Unix)), способных обрабатывать большие объемы данных.

В этом уроке показано, как это сделать и как повторять команды для автоматической обработки любого количества файлов.

Мы продолжим работу в проекте `zipf`, который должен содержать следующие файлы:

```shell
zipf/
└── data
    ├── README.md
    ├── dracula.txt
    ├── frankenstein.txt
    ├── jane_eyre.txt
    ├── moby_dick.txt
    ├── sense_and_sensibility.txt
    ├── sherlock_holmes.txt
    └── time_machine.txt
```

Создадим такую иерархию файлов с помощью Google Colab.

Напомню, что Google Colab - это облачный сервис, предоставляющий интерфейс Jupyter Notebook, который работает на базе операционной системы GNU/Debian и позволяет обращаться к командной оболочке этой операционной системы.

Рассмотрим возможности Google Colab для работы с командной оболочкой.

> Детально про командную оболочку в GNU/Linux по см. [ссылке](https://habr.com/ru/company/ruvds/blog/325522/).







## Запуск команд с помощью символа `!`

Перед командами ставится символ `!`:

In [ ]:
!pwd
!ls

## Запуск команд с помощью %%shell

Магическая команда `%%shell` превращает ячейку блокнота в полноценный файл командной оболочки:

In [ ]:
%%shell

pwd
ls

Существует магическая команда `%shell`, которая превращает строку в командную оболочку:

In [ ]:
%shell pwd

Теперь создадим структуру каталогов.

Удалим созданный ранее каталог `zipf`, если он был в системе:

In [ ]:
%%shell

if [ -d zipf ]; then
rm -rfv zipf
fi

Формируем структуру каталогов:

In [ ]:
%%shell

mkdir zipf
cd zipf
mkdir data
cd data
wget https://raw.githubusercontent.com/dm-fedorov/pandas_basic/master/data/data.zip
unzip data.zip
pwd
ls

## Объединение команд

Чтобы увидеть, как оболочка позволяет нам комбинировать команды, давайте перейдем в каталог `zipf/data` и посчитаем количество строк в каждом файле.

Команда [`wc`](https://www.gnu.org/software/coreutils/manual/coreutils.html#wc-invocation) (сокращение от **w**ord **c**ount) сообщает, сколько строк, слов и букв содержится в одном файле:

In [ ]:
%shell wc zipf/data/moby_dick.txt

Только количество строк (указываем ключ `-l`):

In [ ]:
%shell wc -l zipf/data/moby_dick.txt

Мы можем использовать `wildcard` (подстановочные символы), чтобы сразу указать набор файлов. Чаще всего используется подстановочный символ `*` (одна звездочка). Он соответствует нулю или более символов, поэтому `zipf/data/*.txt` соответствует всем текстовым файлам в каталоге `data`:

In [ ]:
%shell ls zipf/data/*.txt

В то время как `zipf/data/s*.txt` соответствует только двум файлам, имена которых начинаются с `s`:

In [ ]:
%shell ls zipf/data/s*.txt

Подстановочные символы расширяются, чтобы соответствовать именам файлов перед запуском команд, поэтому они работают одинаково для каждой команды. Это означает, что мы можем использовать их с `wc` для подсчета количества слов в книгах с именами, которые содержат подчеркивание:

In [ ]:
%shell wc zipf/data/*_*.txt

Подсчет количества строк в каждом файле:

In [ ]:
%shell wc -l zipf/data/*.txt

Какая из этих книг самая короткая?

Мы можем проверить на глаз, когда файлов всего семь, а если бы их было восемь тысяч?

Наш первый шаг к решению — запустить команду:

In [ ]:
%%shell

cd zipf/data
wc -l *.txt > lengths.txt

Символ "больше" `>` указывает оболочке перенаправить вывод команды в файл, а не печатать его. На экране ничего не появляется; вместо этого все, что могло появиться, ушло в файл `lengths.txt`. Оболочка создает этот файл, если он не существует, или перезаписывает его, если он уже существует.

Мы можем распечатать содержимое файла `lengths.txt`, используя `cat`, что является сокращением от `con cat enate` (потому что, если мы дадим ему имена нескольких файлов, он напечатает их все по порядку):

In [ ]:
%%shell

cat zipf/data/lengths.txt

Теперь мы можем использовать `sort` для сортировки строк в этом файле:

In [ ]:
%%shell

sort -n zipf/data/lengths.txt

На всякий случай мы используем в `sort` опцию `-n`, чтобы указать, что мы хотим сортировать по числам.

`sort` не изменяет `lengths.txt`. Вместо этого она отправляет свой вывод на экран так же, как `wc` ранее. Поэтому мы можем поместить отсортированный список строк в другой временный файл `sorted-lengths.txt` с помощью `>`:

In [ ]:
%%shell

cd zipf/data
sort -n lengths.txt > sorted-lengths.txt

Создание промежуточных файлов с именами типа `lengths.txt` и `sorted-lengths.txt` работает, но отслеживать эти файлы и очищать их, когда они больше не нужны, — утомительное занятие.

Давайте удалим два файла, которые мы только что создали:

In [ ]:
%%shell

cd zipf/data
rm lengths.txt sorted-lengths.txt

Мы можем получить тот же результат с меньшим количеством ввода, используя канал (`pipe`):

In [ ]:
%%shell

cd zipf/data
wc -l *.txt | sort -n

Вертикальная черта `|` между командами `wc` и `sort` сообщает оболочке, что мы хотим использовать выходные данные команды слева в качестве входных данных для команды справа.

Выполнение команды с файлом в качестве входных данных имеет четкий поток информации: команда выполняет задачу над этим файлом и выводит результат на экран (рис.  3.1 а).

Однако при использовании каналов (`pipe`) информация иначе передается после первой команды. Вышестоящая по течению данных команда не читает из файла. Вместо этого она считывает вывод команды, находящейся ниже по течению (рис.  3.1 б).

![pipe](https://raw.githubusercontent.com/dm-fedorov/pandas_basic/master/pic/pipe.png)

Рисунок 3.1: Команды, связанные каналом

Мы можем использовать `|` для сборки каналов любой длины. Например, мы можем использовать команду `head`, чтобы получить только первые три строки отсортированных данных, которые показывают нам три самые короткие книги:

In [ ]:
%shell wc -l zipf/data/*.txt | sort -n | head -n 3

Мы всегда можем перенаправить вывод в файл, добавив `> shortest.txt` в конец канала, тем самым сохранив ответ для дальнейшего использования.

На практике большинство Unix-пользователей создавали бы этот конвейер шаг за шагом, как и мы: начиная с одной команды и добавляя другие команды одну за другой, проверяя вывод после каждого изменения. <br>

<table>
     <tr>
        <td><a href="https://t.me/init_python"><img src="https://dfedorov.spb.ru/pandas/logo-telegram.png" width="50" height="50" alt="telegram"></a></td>
        <td>Обсудить публикацию в [<a href="https://t.me/init_python">Telegram-канале</a>]</td>        
     </tr>
</table>  
<br>
Источник: <br>
"Research Software Engineering with Python. Building software that makes research possible" by Damien Irving, Kate Hertweck, Luke Johnston, Joel Ostblom, Charlotte Wickham, Greg Wilson, <a href="https://merely-useful.tech/py-rse/">https://merely-useful.tech/py-rse/</a>, CC-BY 4.0 и MIT License.<p>